In [63]:
import pandas as pd   # import necessary libraries
import nltk

# NLP in Python
# http://www.nltk.org/book/ch01.html

In [2]:
#Import each of the csv files you downloaded from the SSCC into a pandas df
adzuna = pd.read_csv('~/Dropbox/NU/ADVANCED_MODELING/ADZUNA/datasets/Train_rev1.csv') 

In [13]:
adzuna.head(2)

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk


In [66]:
des = adzuna.iloc[0]['FullDescription']

In [67]:
desToken = nltk.word_tokenize(des)

In [70]:
desToken

['Engineering',
 'Systems',
 'Analyst',
 'Dorking',
 'Surrey',
 'Salary',
 '****K',
 'Our',
 'client',
 'is',
 'located',
 'in',
 'Dorking',
 ',',
 'Surrey',
 'and',
 'are',
 'looking',
 'for',
 'Engineering',
 'Systems',
 'Analyst',
 'our',
 'client',
 'provides',
 'specialist',
 'software',
 'development',
 'Keywords',
 'Mathematical',
 'Modelling',
 ',',
 'Risk',
 'Analysis',
 ',',
 'System',
 'Modelling',
 ',',
 'Optimisation',
 ',',
 'MISER',
 ',',
 'PIONEEER',
 'Engineering',
 'Systems',
 'Analyst',
 'Dorking',
 'Surrey',
 'Salary',
 '****K']

In [71]:
nltk.FreqDist(desToken)

Counter({'****K': 2,
         ',': 6,
         'Analysis': 1,
         'Analyst': 3,
         'Dorking': 3,
         'Engineering': 3,
         'Keywords': 1,
         'MISER': 1,
         'Mathematical': 1,
         'Modelling': 2,
         'Optimisation': 1,
         'Our': 1,
         'PIONEEER': 1,
         'Risk': 1,
         'Salary': 2,
         'Surrey': 3,
         'System': 1,
         'Systems': 3,
         'and': 1,
         'are': 1,
         'client': 2,
         'development': 1,
         'for': 1,
         'in': 1,
         'is': 1,
         'located': 1,
         'looking': 1,
         'our': 1,
         'provides': 1,
         'software': 1,
         'specialist': 1})

In [72]:
desFreq = nltk.FreqDist(desToken)
desFreq.most_common(10)

[(',', 6),
 ('Dorking', 3),
 ('Engineering', 3),
 ('Systems', 3),
 ('Surrey', 3),
 ('Analyst', 3),
 ('Modelling', 2),
 ('Salary', 2),
 ('client', 2),
 ('****K', 2)]